In [1]:
import os

from sklearn.model_selection import train_test_split
import matplotlib
import argparse
import random
import numpy as np
import csv
import pandas as pd
from datascience import *
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer
from keras.models import Model
from keras.models import Sequential, load_model
from keras.layers import *
from keras.optimizers import Adam
from keras import Input
from keras.preprocessing.text import Tokenizer
from keras.utils import plot_model
import keras
from keras.backend.tensorflow_backend import set_session
#config = tf.ConfigProto()
# config.gpu_options.allocator_type = 'BFC' #A "Best-fit with coalescing" algorithm, simplified from a version of dlmalloc.
# config.gpu_options.per_process_gpu_memory_fraction = 0.7
#config.gpu_options.allow_growth =True

#set_session(tf.Session(config=config)) 
import re
os.environ['PYTHONHASHSEED'] = '0'
random.seed(1)
np.random.seed(1)
tf.set_random_seed(1)
from keras.callbacks import *

Using TensorFlow backend.


In [2]:
print(tf.test.is_gpu_available())

True


In [3]:
def sanitize(string):
    words = string.split(' ')
    return words


def converter(x):
    try:
        return ' '.join([x.lower() for x in str(x).split() if x not in stop_words])
    except AttributeError:
        return None  # or some other value
    
def whole(x):
    return int(round(x))

whole(1.0)




1

In [4]:
#read in the csv files 

quora = pd.read_csv("quora_duplicate_questions (1).tsv", sep='\t')


quora['question1']=quora['question1'].fillna("")
quora['question2']=quora['question2'].fillna("")


quora['title1_tokenized'] = \
    quora.loc[:, 'question1'] \
         .apply(sanitize)
quora['title2_tokenized'] = \
    quora.loc[:, 'question2'] \
         .apply(sanitize)



quora.head()

,id,qid1,qid2,question1,question2,is_duplicate,title1_tokenized,title2_tokenized
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,"[What, is, the, step, by, step, guide, to, inv...","[What, is, the, step, by, step, guide, to, inv..."
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,"[What, is, the, story, of, Kohinoor, (Koh-i-No...","[What, would, happen, if, the, Indian, governm..."
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,"[How, can, I, increase, the, speed, of, my, in...","[How, can, Internet, speed, be, increased, by,..."
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,"[Why, am, I, mentally, very, lonely?, How, can...","[Find, the, remainder, when, [math]23^{24}[/ma..."
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,"[Which, one, dissolve, in, water, quikly, suga...","[Which, fish, would, survive, in, salt, water?]"


In [5]:
Tb = Table()
list1 = []
list2 = []

with open("quora_duplicate_questions (1).tsv", 'r', encoding='utf-8') as fd:
    rd = csv.reader(fd, delimiter="\t", quotechar='"')
    for row in rd:
        
        list1.append(row) #pairs start at index 1, with id, setence1, sentence2, label, respectively.
     
    for y in range(6):
        temp=[]
        for x in range(1, len(list1)):
            temp.append(list1[x][y])
            
        list2.append(temp)

    for ele in range(len(list2)):
        Tb=Tb.with_columns(list1[0][ele], list2[ele])


Tb.show(5)

id,qid1,qid2,question1,question2,is_duplicate
0,1,2,What is the step by step guide to invest in share market ...,What is the step by step guide to invest in share market?,0
1,3,4,What is the story of Kohinoor (Koh-i-Noor) Diamond?,What would happen if the Indian government stole the Koh ...,0
2,5,6,How can I increase the speed of my internet connection w ...,How can Internet speed be increased by hacking through DNS?,0
3,7,8,Why am I mentally very lonely? How can I solve it?,Find the remainder when [math]23^{24}[/math] is divided ...,0
4,9,10,"Which one dissolve in water quikly sugar, salt, methane ...",Which fish would survive in salt water?,0


In [31]:
ppdb_Tb = Table()
list1 = []
list2 = []

with open("dataset/ppdb/sanitized.csv") as fd:
    rd = csv.reader(fd)
    for row in rd:
        
        list1.append(row) 
     
    for y in range(6):
        temp=[]
        for x in range(1, len(list1)):
            temp.append(list1[x][y])
            
        list2.append(temp)

    for ele in range(len(list2)):
        ppdb_Tb=ppdb_Tb.with_columns(list1[0][ele], list2[ele])


ppdb_Tb.show(5)

LHS,PHRASE,PARAPHRASE,(FEATURE=VALUE),ALIGNMENT,ENTAILMENT
[VP/NNP],proposed by the president of the,proposed by the chairman of the,PPDB2.0Score=6.13443 PPDB1.0Score=5.794750 -logp(LHS|e1 ...,0-0 1-1 2-2 3-3 4-4 5-5,Equivalence
[VP/NNP],proposed by the chairman of the,proposed by the president of the,PPDB2.0Score=6.11747 PPDB1.0Score=5.794750 -logp(LHS|e1 ...,0-0 1-1 2-2 3-3 4-4 5-5,Equivalence
[VP],referred to in this report,referred to in the present report,PPDB2.0Score=6.04785 PPDB1.0Score=4.678690 -logp(LHS|e1 ...,0-0 1-1 2-2 3-3 3-4 4-5,Equivalence
[VP/NNP],addressed to the president of the,addressed to the chairman of the,PPDB2.0Score=6.03040 PPDB1.0Score=1.919110 -logp(LHS|e1 ...,0-0 1-1 2-2 3-3 4-4 5-5,ReverseEntailment
[VP/NNP],addressed to the chairman of the,addressed to the president of the,PPDB2.0Score=6.01345 PPDB1.0Score=1.919110 -logp(LHS|e1 ...,0-0 1-1 2-2 3-3 4-4 5-5,ForwardEntailment


In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
import nltk
from nltk.corpus import stopwords
stop_words=(stopwords.words('english'))
stop_words=np.append(stop_words, "i'm")
print(stop_words)

In [ ]:
stop_words_punct=np.append(stop_words, "?")


In [ ]:

from scipy import spatial

def cos_sim(vec1, vec2):
    return (1 - spatial.distance.cosine(vec1, vec2))

In [ ]:
import difflib

case_a = 'i like apple'
case_b = 'i like pie'

output_list = [li for li in difflib.ndiff(case_a, case_b) if li[0] != ' ']

In [7]:
from collections import deque

In [8]:
def diff(sen1, sen2):   
    a=[]
    b=[]
    c=[]
    wordlist1 = sen1.split()
    wordlist2 = sen2.split()   
    for i in wordlist1:
        if i.lower() in stop_words:  #omit stop words
            continue    
        if i not in wordlist2:
            a=np.append(a, i)
        else:
            c=np.append(c, i)
    for j in wordlist2:
        if j.lower() in stop_words:   #omit stop words
            continue
        if j not in wordlist1:
            b=np.append(b, j)  
    return np.array(a),np.array(b),np.array(c)       


def seqDiff(sen1,sen2,a,b):      #include length 1
    #wordlist1 = sen1.split()
    #wordlist2 = sen2.split()
    sen1=[x.lower() for x in sen1]
    sen2=[x.lower() for x in sen2]
    
    
    if len(sen1)==1:
                
        if sen1[0] in sen2 or sen1[0] in stop_words:
            if len(a)>0:
                b.append(a)
                
            return b
        
        else:
            a=np.append(a,sen1[0])
            b.append(a)
                
            return b
    
    elif sen1[0] in sen2:
        
        if len(a)>0:
            b.append(a)
        a=[]        
        return seqDiff(sen1[1:], sen2,a,b)
        
    elif (sen1[0] not in sen2):
        
        if sen1[0] not in stop_words:
            a=np.append(a,sen1[0])
        
        elif len(a)>0: 
            b.append(a)
            a=[]
            
        return seqDiff(sen1[1:], sen2,a,b)
    
    
    
def seqDiff2(sen1,sen2,a,b):     # inlude length 1
    sen1=[x.lower() for x in sen1]
    sen2=[x.lower() for x in sen2]
    
    if len(sen1)==1:
                
        if sen1[0] in sen2:
            if len(a)>0:
                b.append(a)
                
            return b
        
        else:
            a=np.append(a,sen1[0])
            b.append(a)
                
            return b
    
    elif sen1[0] in sen2:
        if len(a)>0:
            b.append(a)
        a=[]        
        return seqDiff2(sen1[1:], sen2,a,b)
        
    elif (sen1[0] not in sen2):
        
        a=np.append(a,sen1[0])
        
                   
        return seqDiff2(sen1[1:], sen2,a,b)
                

In [ ]:
g="How can I be a good geologist?" 
hue="What should I do to be a great geologist"
g=g.split()
hue=hue.split()
g=[x.lower() for x in g]
hue=[x.lower() for x in hue]
deck=deque()
temp=seqDiff2(g,hue,[],deck)
temp

In [ ]:
a[0]

In [ ]:
output_list

In [14]:
true_Tb = Tb.where("is_duplicate", are.equal_to("1"))
true_Tb.show(30)

id,qid1,qid2,question1,question2,is_duplicate
5,11,12,Astrology: I am a Capricorn Sun Cap moon and cap rising. ...,"I'm a triple Capricorn (Sun, Moon and ascendant in Capri ...",1
7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
11,23,24,How do I read and find my YouTube comments?,How can I see all my Youtube comments?,1
12,25,26,What can make Physics easy to learn?,How can you make physics easy to learn?,1
13,27,28,What was your first sexual experience like?,What was your first sexual experience?,1
15,31,32,What would a Trump presidency mean for current internati ...,How will a Trump presidency affect the students presentl ...,1
16,33,34,What does manipulation mean?,What does manipulation means?,1
18,37,38,Why are so many Quora users posting questions that are r ...,Why do people ask Quora questions which can be answered ...,1
20,41,42,Why do rockets look white?,Why are rockets and boosters painted white?,1
29,59,60,How should I prepare for CA final law?,How one should know that he/she completely prepare for C ...,1


In [21]:
row = true_Tb.num_rows
sent1 =  true_Tb.column("question1")
sent2 = true_Tb.column("question2")
a=[0]*row
b=[0]*row

bleh=deque()

for x in range((row)):
    sen1=no_ques(sent1[x])
    sen2=no_ques(sent2[x])
    sen1=sen1.split()
    sen2=sen2.split()
    
    d = seqDiff2(sen1, sen2, bleh, [])
    e = seqDiff2(sen2,sen1,bleh,[])
    a[x]=d
    b[x]=e    
   



In [20]:
def no_ques(sent1):
    
    g = [x.replace("?", "") for x in sent1]
    
    return ''.join(g)

In [ ]:
new_Tb=Table().with_column( "sent1_only", a, "sent2_only",b)

In [ ]:
new_Tb.to_csv("sanitized_QQP_dataset.csv")

In [ ]:
new_Tb.show(40) 

In [41]:
row = Tb.num_rows
id = Tb.column("id")

s1 =  Tb.column("question1")
s2 = Tb.column("question2")  #qqp
phrase = ppdb_Tb.column("PHRASE")
paraphrase=ppdb_Tb.column("PARAPHRASE")   #ppdb
label = Tb.column("is_duplicate")

In [ ]:

index_list=[]
PPDB_id = []
sent1=[]
sent2=[]
para_list1=[]
para_list2=[]
labeling = []

ans = []
for x in range(len(s1)):   #index of QQP
    
    for z in range(len(phrase)):  #index of PPDB
        if (phrase[z] in s1[x] and paraphrase[z] in s2[x]):
            print(phrase[z])
            index_list=np.append(index_list, x)
            PPDB_id=np.append(PPDB_id, z)
            sent1=np.append(sent1, s1[x])
            sent2=np.append(sent2, s2[x])
            para_list1=np.append(para_list1, phrase[z])
            para_list2=np.append(para_list2, paraphrase[z])
            labeling = np.append(labeling, label[x])
         

        
        
   
        

 the types of 
 the types of 
 types of 
 the types 
 most powerful 
 most people 
 matter whether 
 how many 
 many different 
 different things 
 the effects of 
 effects of 
 the effects 
 enough to 
 mental illness 
 most important 
 most important 
 are the most important 
 friends 
 70 years 
 too late 
 too late 
 climate change 
 higher education 
 feel like 
 daily life 
 solar energy 
 solar 
 friendly relations 
 is the meaning of 
 invest in 
 pay off 
 find love 
 services in 
 services in 
 best available 
 best available 
 the best available 
 is continuing 
 continuing in 
 run away 
 run away 
 most beautiful 
 most amazing 
 amazing 
 is most important 
 most important 
 is most important 
 is most important 
 most important 
 most important 
 most important 
 education system 
 education system 
 too late 
 too late 
 management information 
 information systems 
 weather conditions 
 left behind 
 feel like 
 still exist 


In [48]:
phrase_dataset = Table().with_column("QQP_id", index_list, "PPDB_id", PPDB_id, "sent1", sent1, "sent2", sent2, "paraphrase1", para_list1, "paraphrase2", para_list2, "label", labeling)

In [49]:
phrase_dataset.show(5)

QQP_id,PPDB_id,sent1,sent2,paraphrase1,paraphrase2,label
70,641086,What are the types of immunity?,What are the different types of immunity in our body?,the types of,the different types of,0
70,641090,What are the types of immunity?,What are the different types of immunity in our body?,the types of,the different types of,0
70,894710,What are the types of immunity?,What are the different types of immunity in our body?,types of,the different types of,0
70,1.49785e+06,What are the types of immunity?,What are the different types of immunity in our body?,the types,the different types,0
111,173763,Is USA the most powerful country of the world?,Why is the USA the most powerful country of the world?,most powerful,powerful,0


In [50]:
phrase_dataset2=phrase_dataset.to_df()
phrase_dataset2.to_csv("dataset/Phrase_Level/QQP/PPDB_QQP.tsv", sep='\t', index=False)

In [51]:
phrase_dataset.num_rows

20904

In [ ]:
true_Tb.to_csv("QQP_Trues")

In [ ]:
#create a dictionary

MAX_NUM_WORDS = 20000
tokenizer = keras \
    .preprocessing \
    .text \
    .Tokenizer(num_words=MAX_NUM_WORDS)

corpus_x1 = quora.title1_tokenized
corpus_x2 = quora.title2_tokenized
corpus = pd.concat([
  corpus_x1, corpus_x2])
#corpus.shape
tokenizer.fit_on_texts(corpus)
x1 = tokenizer \
    .texts_to_sequences(corpus_x1)
x2 = tokenizer \
    .texts_to_sequences(corpus_x2)

word_index=tokenizer.word_index




corpus.head()

In [ ]:
max_seq_len = max([            #why does seq length change when I change dictionary size?
    len(seq) for seq in x1])
max_seq_len

In [ ]:
max_seq_len1 = max([
    len(seq) for seq in x1])
print(max_seq_len1)   

MAX_SEQUENCE_LENGTH = 115  #better to have words covered than uncovered
x1 = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(x1, 
                   maxlen=MAX_SEQUENCE_LENGTH)


x2 = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(x2, 
                   maxlen=MAX_SEQUENCE_LENGTH)


x1[:5]




In [ ]:


dup = quora.is_duplicate

y = keras \
    .utils \
    .to_categorical(dup)



print(len(quora))

y[:8]

In [ ]:
import math


def foo():
    q=[]
    p=0
    for x in quora.is_duplicate:
        if math.isnan(x):
            q = np.append(q, p)
        p+=1
    return q
foo()

In [ ]:
# We randomly select 5,000 paraphrases and 5,000 non-paraphrases as the dev set, and sample
#another 5,000 paraphrases and 5,000 non-paraphrases as the
#test set. We keep the remaining instances as the training set 2

label = quora.is_duplicate
eq_list = []
uneq_list=[]

count = 0

for x in range(len(quora.is_duplicate)):
    if label[x]==1:
        eq_list = np.append(eq_list, x) 
    
    else:
        uneq_list = np.append(uneq_list, x)

        
eq_list[:3]



In [ ]:
eq_list=eq_list.astype(int)
uneq_list=uneq_list.astype(int)

In [ ]:
import random

random.shuffle(eq_list)
random.shuffle(uneq_list)

eq_list[:10]


In [ ]:
index_test_eq = eq_list[:5000]

index_test_uneq = uneq_list[:5000]

index_dev_eq = eq_list[5000:10000]

index_dev_uneq = uneq_list[5000:10000]

index_train_eq = eq_list[10000:]

index_train_uneq = uneq_list[10000:]

index_test = np.append(index_test_eq, index_test_uneq)

index_dev = np.append(index_dev_eq, index_dev_uneq)

index_train = np.append(index_train_eq, index_train_uneq)

random.shuffle(index_test)
random.shuffle(index_dev)
random.shuffle(index_train)



In [ ]:


x1_test = [x1[i] for i in index_test]

x2_test = [x2[i] for i in index_test]

y_test = [y[i] for i in index_test]


x1_dev = [x1[i] for i in index_dev]

x2_dev = [x2[i] for i in index_dev]

y_dev=[y[i] for i in index_dev]


x1_train = [x1[i] for i in index_train]

x2_train = [x2[i] for i in index_train]

y_train=[y[i] for i in index_train]


print("Training Set")
print("-" * 10)
print(np.shape(x1_train))
print(np.shape(x2_train))
print(np.shape(y_train))


print("-" * 10)
print(np.shape(x1_test))
print(np.shape(x2_test))
print(np.shape(y_test))


print("Test Set")
print(y_test[:5])

In [ ]:
#add cos similarity value to last elem of word embedding
x1_test_cos=[]
x2_test_cos=[]
x1_train_cos=[]
x2_train_cos=[]
x1_dev_cos=[]
x2_dev_cos=[]

from scipy import spatial

def cos_sim(vec1, vec2):
    return (1 - spatial.distance.cosine(vec1, vec2))

for x in range(len(y_train)):
    
    
    cos_train=cos_sim(x1_train[x], x2_train[x])
    
    
    x1_train_cos.append(np.append(x1_train[x], cos_train))
    x2_train_cos.append(np.append(x2_train[x], cos_train))
    
    


for x in range(len(y_test)):
    cos_dev=cos_sim(x1_dev[x], x2_dev[x])
    cos_test=cos_sim(x1_test[x], x2_test[x])
    x1_test_cos.append(np.append(x1_test[x], cos_test))
    x2_test_cos.append(np.append(x2_test[x], cos_test))
    x1_dev_cos.append(np.append(x1_dev[x], cos_dev))
    x2_dev_cos.append(np.append(x2_dev[x], cos_dev))

In [ ]:
np.array(x1_test_cos)[:10]
len(x1_train_cos[1])

In [ ]:
#BiLSTM with GLoVE 


NUM_CLASSES=2 #boolean; 1 or 0

MAX_NUM_WORDS = 20000 #rough estimate

MAX_SEQUENCE_LENGTH = 116 #how long one sentence is

NUM_EMBEDDING_DIM = 300 #this is arbitrary?

NUM_LSTM_UNITS = 128 #output dimension




top_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ),    #this is the first sentence
    dtype='int32')

bm_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ),   #this is the second
    dtype='int32')



embedding_layer = embedding_layer = Embedding(len(word_index)+1,
                            NUM_EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)
top_embedded = embedding_layer(
    top_input)
bm_embedded = embedding_layer(
    bm_input)


shared_lstm = Bidirectional(LSTM(NUM_LSTM_UNITS))
top_output = shared_lstm(top_embedded)
bm_output = shared_lstm(bm_embedded)


merged = concatenate(
    [top_output, bm_output], 
    axis=-1)


dense =  Dense(
    units=NUM_CLASSES, 
    activation='sigmoid')
predictions = dense(merged)



model = Model(
    inputs=[top_input, bm_input], 
    outputs=predictions)


model.summary()

model.compile(
    optimizer='Adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

BATCH_SIZE = 1024

NUM_EPOCHS = 15

earlyStopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='min')
mcp_save = ModelCheckpoint('quora_weights_with_GLoVE.h5', verbose=1, save_best_only=True, monitor='val_loss', mode='min')


In [ ]:
#BiLSTM with last cos_sim element concatenated?
NUM_CLASSES=2 #boolean; 1 or 0

#rough estimate

MAX_SEQUENCE_LENGTH = 51 #how long one sentence is

#this is arbitrary?

NUM_LSTM_UNITS = 128 #output dimension


NUM_EMBEDDING_DIM = 256

top_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ),    #this is the first sentence
    dtype='int32')

bm_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ),   #this is the second
    dtype='int32')



embedding_layer = Embedding(
    MAX_NUM_WORDS, NUM_EMBEDDING_DIM)

top_embedded = embedding_layer(
    top_input)
bm_embedded = embedding_layer(
    bm_input)



shared_lstm = Bidirectional(LSTM(NUM_LSTM_UNITS))

top_output = shared_lstm(top_embedded)

bm_output = shared_lstm(bm_embedded)




merged = concatenate(
    [top_output, bm_output], 
    axis=-1)


dense =  Dense(
    units=NUM_CLASSES, 
    activation='sigmoid')
predictions = dense(merged)



model = Model(
    inputs=[top_input, bm_input], 
    outputs=predictions)


model.summary()

model.compile(
    optimizer='Adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

BATCH_SIZE = 512

NUM_EPOCHS = 30

earlyStopping = EarlyStopping(monitor='val_acc', patience=10, verbose=1, mode='max')
mcp_save = ModelCheckpoint('BiLSTM_PAWS_QQP_concat_sim_cos.h5', save_best_only=True,  verbose=1, monitor='val_acc', mode='max')


In [ ]:

history = model.fit(x=([np.array(x1_train_cos), np.array(x2_train_cos)]), y=np.array(y_train), batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, verbose=1, callbacks=[earlyStopping, mcp_save], validation_data=([np.array(x1_dev_cos), np.array(x2_dev_cos)], np.array(y_dev)),
    
    shuffle=True
)

In [ ]:

history = model.fit(x=([np.array(x1_train), np.array(x2_train)]), y=np.array(y_train), batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, verbose=1, callbacks=[earlyStopping, mcp_save], validation_data=([np.array(x1_dev), np.array(x2_dev)], np.array(y_dev)),
    
    shuffle=True
)

In [ ]:
model = load_model("quora_weights_with_GLoVE.h5")



In [ ]:
predicts = model.predict(
    [x1_test, x2_test])

In [ ]:
one = Tb.column('question1')
two = Tb.column('question2')
no = []
err1 = []
err2 = []
wrongCat = []


for x in range(len(index_test)):
    if (np.argmax(predicts[x])!=(np.argmax(y_test[x]))):
        no=np.append(no, index_test[x])
        err1=np.append(err1, one[index_test[x]])
        err2=np.append(err2, two[index_test[x]])
        wrongCat = np.append(wrongCat, np.argmax(y_test[x]))
        
print(1-(len(wrongCat)/len(x1_test)))

In [ ]:
error_analysis = Table()


error_analysis=error_analysis.with_column("id", no, "sent1", err1, "sent2", err2, "classification", wrongCat)
error_analysis.show(30)
           


In [ ]:
hue = error_analysis.where("classification", are.equal_to(0)) 
hue


In [ ]:


print('Found %s word vectors.' % len(glove))

NUM_EMBEDDING_DIM = 300

embedding_matrix = np.zeros((len(word_index)+1, NUM_EMBEDDING_DIM))


for word, i in word_index.items():
    embedding_vector = glove.get(word)
    
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [ ]:
def process_glove_line(line, dim):
    word = None
    embedding = None

    try:
        splitLine = line.split()
        word = " ".join(splitLine[:len(splitLine)-dim])
        embedding = np.array([float(val) for val in splitLine[-dim:]])
    except:
        print(line)

    return word, embedding

def load_glove_model(glove_filepath, dim):
    with open(glove_filepath, encoding="utf8" ) as f:
        content = f.readlines()
        model = {}
        for line in content:
            word, embedding = process_glove_line(line, dim)
            if embedding is not None:
                model[word] = embedding
        return model

glove = load_glove_model("glove.840B.300d.txt", 300)

In [ ]:
model = load_model('BiLSTM_QQP.h5')